In [ ]:
# lanchain RAG示例

In [6]:
import getpass
import os
AI_KEY = "sk-Swi6dHHVWDY342vVaCwFLwmguz6YXfVlSXAfNxzukMtsScfP"
AI_URL = "https://api.chatanywhere.tech/v1"

os.environ["OPENAI_API_KEY"] = AI_KEY
os.environ["OPENAI_API_BASE"] = AI_URL
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [7]:
import bs4
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

In [8]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [9]:
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [10]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

print(question_answer_chain)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

print(rag_chain)

bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), config={'run_name': 'format_inputs'})
| ChatPromptTemplate(input_variables=['chat_history', 'context', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, say that you don't know. Use three sentences maximum and keep the answer concise.\n\n{context}")), MessagesPlaceholder(variable_name='chat_history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'

In [11]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [12]:
conversational_rag_chain.invoke(
    {"input": "What is Task Decomposition?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

Parent run 24044ac8-223a-4530-8db1-3adea5c8e061 not found for run 10742951-1829-4df6-8ca0-16de83d45ab9. Treating as a root run.


'Task decomposition is a technique used to break down complex tasks into smaller and more manageable steps. It helps agents or models handle intricate tasks by dividing them into simpler subtasks. This process can be facilitated by various methods such as prompting with specific instructions, human inputs, or utilizing language models with appropriate prompts like "Steps for XYZ."'

In [ ]:
# langchain schema规范示例，可以解析想要的输出

In [13]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser


gift_response_schema = ResponseSchema(name="gift_response", description="A gift response")
delight_response_schema = ResponseSchema(name="delight_response", description="A delight response")
price_response_schema = ResponseSchema(name="price_response", description="A price response")

response_schema=[gift_response_schema, delight_response_schema, price_response_schema]
output_parser = StructuredOutputParser.from_response_schemas(response_schema)



In [14]:
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift_response": string  // A gift response
	"delight_response": string  // A delight response
	"price_response": string  // A price response
}
```


In [ ]:
# langchain Buffermemory示例,可以存储和读取数据,实现对话上下文联系
# BufferMemory是一个简单的内存存储器，用于存储和检索数据。它是一个键值存储器，其中键是字符串，值是任何Python对象。

In [15]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain_openai import ChatOpenAI
import getpass
import os
AI_KEY = "sk-Swi6dHHVWDY342vVaCwFLwmguz6YXfVlSXAfNxzukMtsScfP"
AI_URL = "https://api.chatanywhere.tech/v1"

os.environ["OPENAI_API_KEY"] = AI_KEY
os.environ["OPENAI_API_BASE"] = AI_URL
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
memory=ConversationBufferMemory()

conversation = ConversationChain(
    llm=llm, verbose=True, memory=memory
)



In [16]:
conversation.predict(input="Hi there!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there!
AI:

> Finished chain.


'Hello! How can I assist you today?'

In [29]:
conversation.predict(input="What is Task Decomposition?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: What is Task Decomposition?
AI:

> Finished chain.


'Task Decomposition is a strategy used in project management and software development to break down complex tasks into smaller, more manageable sub-tasks. This approach helps teams to better understand the requirements of a project, allocate resources effectively, and track progress more efficiently. By decomposing tasks, teams can identify dependencies, prioritize work, and improve overall productivity. It is a common practice in agile methodologies such as Scrum and Kanban.'

In [31]:
print(memory.buffer)

Human: What is Task Decomposition?
AI: Task Decomposition is a strategy used in project management and software development to break down complex tasks into smaller, more manageable sub-tasks. This approach helps teams to better understand the requirements of a project, allocate resources effectively, and track progress more efficiently. By decomposing tasks, teams can identify dependencies, prioritize work, and improve overall productivity. It is a common practice in agile methodologies such as Scrum and Kanban.
Human: Hi there!
AI: Hello! How can I assist you today?


In [33]:
from langchain.memory import ConversationBufferWindowMemory
from langchain.memory import ConversationTokenBufferMemory
from langchain.memory import ConversationSummaryBufferMemory

In [ ]:
memory=ConversationBufferWindowMemory(k=3)
# k 表示存储的对话数量,从最近的对话开始存储

In [ ]:
memory=ConversationTokenBufferMemory(max_token_limit=10)
# 这个内存存储器将存储最多10个令牌的对话，从最近的对话开始存储

In [34]:
memory=ConversationSummaryBufferMemory(llm=llm, max_tokens_limit=400)  
# 这个内存存储器将存储最多400个令牌的对话，可以一直加入新的记忆，但是会调用LLM，将存进来的记忆变成摘要，以节省空间，不超出400个令牌的限制

In [ ]:
# lanchain强大的链:LLMChain

In [55]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
llm=ChatOpenAI()

In [72]:
prompt=ChatPromptTemplate.from_template("Ask a question about Task Decomposition.{input}")

In [75]:
chain=LLMChain(llm=llm, prompt=prompt, verbose=True)

In [76]:
chain.run("what is Task Decomposition?")



> Entering new LLMChain chain...
Prompt after formatting:
Human: Ask a question about Task Decomposition.what is Task Decomposition?

> Finished chain.


'Task decomposition is a process of breaking down a complex task or project into smaller, more manageable subtasks or steps in order to facilitate better planning, organization, and execution. This approach helps to identify all the necessary components of a task, assign responsibilities to team members, and ensure that the overall goal is achieved efficiently and effectively.'

In [ ]:
# SequentialChain,可以将多个链连接在一起，形成一个链条

In [59]:
from langchain.chains import SimpleSequentialChain
# simple链在处理一个输出的时候效果不错

In [68]:
llm = ChatOpenAI()
first_prompt = ChatPromptTemplate.from_template("Ask a question about Task Decomposition.{DELIM}")
chain_one=LLMChain(llm=llm, prompt=first_prompt)

In [69]:
second_prompt = ChatPromptTemplate.from_template("Ask a question about Task Decomposition.{DELIM}")
chain_two=LLMChain(llm=llm, prompt=second_prompt)

In [70]:
over_all_chain=SimpleSequentialChain(chains=[chain_one, chain_two],verbose=True)

In [71]:
over_all_chain.run("what is Task Decomposition?")



> Entering new SimpleSequentialChain chain...
Task decomposition is a strategy used in project management and problem-solving where a complex task or project is broken down into smaller, more manageable sub-tasks or components. This allows for better organization, delegation of responsibilities, and overall efficiency in completing the task.
How can task decomposition help improve the efficiency and effectiveness of project management?

> Finished chain.


'How can task decomposition help improve the efficiency and effectiveness of project management?'